In [0]:
%pip install -qqqq -U -r requirements.txt

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.1.20 requires langchain-core<0.2.0,>=0.1.52, but you have langchain-core 0.3.25 which is incompatible.
langchain 0.1.20 requires langsmith<0.2.0,>=0.1.17, but you have langsmith 0.2.3 which is incompatible.
langchain-community 0.0.38 requires langchain-core<0.2.0,>=0.1.52, but you have langchain-core 0.3.25 which is incompatible.
langchain-community 0.0.38 requires langsmith<0.2.0,>=0.1.0, but you have langsmith 0.2.3 which is incompatible.
langchain-text-splitters 0.0.2 requires langchain-core<0.3,>=0.1.28, but you have langchain-core 0.3.25 which is incompatible.
ydata-profiling 4.5.1 requires pandas!=1.4.0,<2.1,>1.1, but you have pandas 2.2.3 which is incompatible.
ydata-profiling 4.5.1 requires pydantic<2,>=1.8.1, but you have pydantic 2.10.3 which is incompatible.
Note: you may need to restart the

In [0]:
dbutils.library.restartPython()

In [0]:
from cookbook.tools.vector_search import (
    VectorSearchRetrieverTool,
    VectorSearchSchema,
)
from cookbook.tools.uc_tool import UCTool

Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x7ff9cf400ae0>
Traceback (most recent call last):
  File "/databricks/python/lib/python3.11/site-packages/threadpoolctl.py", line 400, in match_module_callback
    self._make_module_from_path(filepath)
  File "/databricks/python/lib/python3.11/site-packages/threadpoolctl.py", line 515, in _make_module_from_path
    module = module_class(filepath, prefix, user_api, internal_api)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/databricks/python/lib/python3.11/site-packages/threadpoolctl.py", line 606, in __init__
    self.version = self.get_version()
                   ^^^^^^^^^^^^^^^^^^
  File "/databricks/python/lib/python3.11/site-packages/threadpoolctl.py", line 646, in get_version
    config = get_config().split()
             ^^^^^^^^^^^^^^^^^^
AttributeError: 'NoneType' object has no attribute 'split'


In [0]:
retriever_tool = VectorSearchRetrieverTool(
    name="search_product_docs",
    description="Use this tool to search for product documentation.",
    vector_search_index="dbdemos.dbdemos_rag_chatbot.databricks_documentation_vs_index",
    vector_search_schema=VectorSearchSchema(
        # These columns are the default values used in the `01_data_pipeline` notebook
        # If you used a different column names in that notebook OR you are using a pre-built vector index, update the column names here.
        chunk_text="content",  # Contains the text of each document chunk
        document_uri="url",  # The document URI of the chunk e.g., "/Volumes/catalog/schema/volume/file.pdf" - displayed as the document ID in the Review App
        # additional_metadata_columns=[],  # Additional columns to return from the vector database and present to the LLM
    )
)


translate_sku_tool = UCTool(uc_function_name="devanshu_pandey.cmhc_demo.vector_index_search_tool")

tools = [retriever_tool]

In [0]:
entry_point = dbutils.notebook.entry_point

host_name = f'https://{entry_point.getDbutils().notebook().getContext().browserHostName().get()}'

token = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get()

In [0]:
base_url = f"{host_name}/serving-endpoints/"
base_url

'https://adb-984752964297111.11.azuredatabricks.net/serving-endpoints/'

In [0]:
def is_termination_message(message):
    content = message.get("content", "")
    return (content and "TERMINATE" in content.upper()) or (message['role'] == 'user' and 'tool_calls' not in message)

In [0]:
from databricks.sdk import WorkspaceClient
from databricks.sdk.service.serving import ChatMessage, ChatMessageRole
from typing import List, Optional


class DatabricksModelServingClient:
    def __init__(self, config, **kwargs):
        self.workspace = WorkspaceClient()
        self.openai_client = self.workspace.serving_endpoints.get_open_ai_client()
        self.endpoint_name = config.get("endpoint_name")
        self.llm_config = config.get("llm_config")

    def create(self, input_data):
      messages = []
      for message in input_data['messages']:
        message.pop("name", None)
        messages.append(message)

      response = self.openai_client.chat.completions.create(
          model=self.endpoint_name,
          messages=messages,
          tools=input_data['tools'],
          tool_choice="auto",
          **self.llm_config
      )
      
      return response

    def message_retrieval(self, response):
      # Process and return messages from the response
      return [choice.message for choice in response.choices]

    def cost(self, response):
      # Implement cost calculation if applicable
      return 0

    def get_usage(self, response):
      usage = response.usage
      # Implement usage statistics if available
      return {"prompt_tokens": usage.prompt_tokens, "total_tokens": usage.total_tokens, "completion_tokens": usage.completion_tokens}

In [0]:
config_list = {
    "model_client_cls": "DatabricksModelServingClient",
    "model": "gpt4o",
    "endpoint_name": "casaman-gpt4", # "databricks-meta-llama-3-3-70b-instruct",
    "llm_config": {"temperature": 0.5, "max_tokens": 1500}
    
}

In [0]:
import os

from autogen import ConversableAgent

def create_agents(system_prompt, chat_history):

    def is_termination_message(message):
        content = message.get("content", "")
        return (content and "TERMINATE" in content.upper()) or (message['role'] == 'user' and 'tool_calls' not in message)

    # The user proxy agent is used for interacting with the assistant agent
    # and executes tool calls.
    user_proxy = ConversableAgent(
        name="User",
        llm_config=False,
        is_termination_msg=is_termination_message,
        human_input_mode="NEVER",
    )

    assistant = ConversableAgent(
        name="Assistant",
        system_message="You are a helpful AI assistant. "
        "You can help with simple calculations. "
        "Return 'TERMINATE' when the task is done.",
        llm_config={"config_list": [config_list]},
        chat_messages={user_proxy: chat_history}
    )

    return assistant, user_proxy
    
assistant, user_proxy = create_agents('test', [])

[autogen.oai.client: 12-17 20:21:39] {509} INFO - Detected custom model client in config: DatabricksModelServingClient, model client can not be used until register_model_client is called.


INFO:autogen.oai.client:Detected custom model client in config: DatabricksModelServingClient, model client can not be used until register_model_client is called.


In [0]:
from autogen import register_function

for tool in tools:
  register_function(
      tool,
      caller=assistant,  # The assistant agent can suggest calls to the calculator.
      executor=user_proxy,  # The user proxy agent can execute the calculator calls.
      name=tool.name,
      description=tool.description,  # A description of the tool.
  )

[autogen.oai.client: 12-17 20:21:41] {509} INFO - Detected custom model client in config: DatabricksModelServingClient, model client can not be used until register_model_client is called.


INFO:autogen.oai.client:Detected custom model client in config: DatabricksModelServingClient, model client can not be used until register_model_client is called.


In [0]:
translate_sku_tool._toolkit.tools[0].register_function(callers = assistant,
                                executors = user_proxy )

[autogen.oai.client: 12-17 20:21:43] {509} INFO - Detected custom model client in config: DatabricksModelServingClient, model client can not be used until register_model_client is called.


INFO:autogen.oai.client:Detected custom model client in config: DatabricksModelServingClient, model client can not be used until register_model_client is called.


In [0]:
assistant.register_model_client(model_client_cls=DatabricksModelServingClient)

In [0]:
chat_result = user_proxy.initiate_chat(assistant, message="What is mlflow in databricks?")

User (to Assistant):

What is mlflow in databricks?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Assistant (to User):

***** Suggested tool call (call_RcySHcYfv5iSQcboahKj0tu5): search_product_docs *****
Arguments: 
{"query":"mlflow in databricks"}
************************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION search_product_docs...
User (to Assistant):

User (to Assistant):

***** Response from calling tool (call_RcySHcYfv5iSQcboahKj0tu5) *****
[{"page_content": "ML lifecycle management using MLflow  \nThis article describes how MLflow is used in Databricks for machine learning lifecycle management. It also includes examples that introduce each MLflow component and links to content that describe how these components are hosted within Databricks.  \nML lifecycle management in Databricks is

Trace(request_id=tr-233b2640bd5e45ed90347d540d53c7e2)

In [0]:
assistant.last_message(user_proxy)

{'content': "MLflow is an open-source platform used within Databricks for managing the end-to-end machine learning lifecycle. It offers several key components to facilitate this:\n\n1. **Tracking**: It allows you to track experiments and record metrics related to machine learning models, enabling you to compare parameters and results across different runs.\n   \n2. **Models**: It supports the management and deployment of models from various ML libraries to different serving and inference platforms, allowing for a broad usage across different environments.\n\n3. **Projects**: Allows ML code to be packaged in a reusable and reproducible manner so that it can be shared with other data scientists or transferred into production environments.\n\n4. **Model Registry**: This component centralizes a model store to manage the full lifecycle of models, from training and validation to deployment and monitoring.\n\nDatabricks provides a fully managed and hosted version of MLflow that is integrated 

In [0]:
history = assistant.chat_messages[user_proxy]
history

  File <command-3722098005799721>, line 5
    "llm_config": {"temperature": 0.5, "max_tokens": 1500}
                ^
SyntaxError: invalid syntax


In [0]:
assistant, user_proxy = create_agents('test', history)

  File <command-3722098005799721>, line 5
    "llm_config": {"temperature": 0.5, "max_tokens": 1500}
                ^
SyntaxError: invalid syntax


In [0]:
assistant.chat_messages[user_proxy]

  File <command-3722098005799721>, line 5
    "llm_config": {"temperature": 0.5, "max_tokens": 1500}
                ^
SyntaxError: invalid syntax


In [0]:
chat_result = user_proxy.initiate_chat(assistant, message="This is the second turn of the conversation. Can you summary our actual conversation?", clear_history=False)

  File <command-3722098005799721>, line 5
    "llm_config": {"temperature": 0.5, "max_tokens": 1500}
                ^
SyntaxError: invalid syntax


In [0]:
assistant.llm_config["tools"]

  File <command-3722098005799721>, line 5
    "llm_config": {"temperature": 0.5, "max_tokens": 1500}
                ^
SyntaxError: invalid syntax


In [0]:
assistant.last_message(user_proxy)

  File <command-3722098005799721>, line 5
    "llm_config": {"temperature": 0.5, "max_tokens": 1500}
                ^
SyntaxError: invalid syntax
